In [1]:
!pip install accelerate peft bitsandbytes transformers trl unsloth optree

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.8/410.8 kB 11.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 24.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 102.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 6.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.9/126.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [2]:
import torch
import pandas as pd
import joblib
import warnings
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import (LongformerForSequenceClassification, LongformerTokenizerFast,
                          Trainer, TrainingArguments, DataCollatorWithPadding)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType

warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [3]:
class CFG:
    checkpoint = "allenai/longformer-base-4096"
    max_length = 2048
    per_device_train_batch_size = 8
    per_device_eval_batch_size = 16
    gradient_accumulation_steps = 2
    n_epochs = 3
    lr = 2e-5
    warmup_steps = 20
    lora_r = 8
    lora_alpha = 16
    lora_dropout = 0.1
    lora_bias = "none"
    seed = 42

In [4]:
from sklearn.model_selection import train_test_split

In [13]:
dataset = pd.read_parquet("/kaggle/input/wsdm-cup-multilingual-chatbot-arena/train.parquet").reset_index(drop=True)
dataset["winner"] = dataset["winner"].map({"model_a": 0, "model_b": 1})

train, test = train_test_split(dataset, test_size=0.2, random_state=1)
val, test = train_test_split(test, test_size=0.5, random_state=1)

train = Dataset.from_pandas(train.sample(10000))
val = Dataset.from_pandas(val)
test = Dataset.from_pandas(test)

tokenizer = LongformerTokenizerFast.from_pretrained(CFG.checkpoint)
tokenizer.padding_side = "right"

In [14]:
def encode(batch):
    texts = [f"<prompt>: {p}\n\n<response_a>: {a}\n\n<response_b>: {b}" 
             for p, a, b in zip(batch["prompt"], batch["response_a"], batch["response_b"])]
    tokenized = tokenizer(texts, max_length=CFG.max_length, truncation=True, padding="max_length")
    return {**tokenized, "labels": batch["winner"]}

In [15]:
train = train.map(encode, batched=True)
val = val.map(encode, batched=True)
test = test.map(encode, batched=True)

lora_config = LoraConfig(
    r=CFG.lora_r,
    lora_alpha=CFG.lora_alpha,
    target_modules=["query", "value"],
    lora_dropout=CFG.lora_dropout,
    bias=CFG.lora_bias,
    task_type=TaskType.SEQ_CLS,
)

model = LongformerForSequenceClassification.from_pretrained(
    CFG.checkpoint,
    num_labels=2,
    torch_dtype=torch.float16,
    device_map="auto",
)

model.config.use_cache = False
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4844 [00:00<?, ? examples/s]

Map:   0%|          | 0/4844 [00:00<?, ? examples/s]

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
from transformers import TrainerCallback

In [17]:
def compute_metrics(eval_preds):
    preds = eval_preds.predictions
    labels = eval_preds.label_ids
    probs = torch.from_numpy(preds).float().softmax(-1).numpy()
    loss = log_loss(y_true=labels, y_pred=probs)
    acc = accuracy_score(y_true=labels, y_pred=preds.argmax(-1))
    return {"acc": acc, "log_loss": loss}

class LoggingCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % 10 == 0:  # Log every 10 steps
            if state.log_history and 'loss' in state.log_history[-1]:
                print(f"Step {state.global_step}: Loss = {state.log_history[-1]['loss']}, All = {state.log_history[-1]}")

training_args = TrainingArguments(
    output_dir="longformer_finetuned",
    num_train_epochs=CFG.n_epochs,
    per_device_train_batch_size=CFG.per_device_train_batch_size,
    gradient_accumulation_steps=CFG.gradient_accumulation_steps,
    per_device_eval_batch_size=CFG.per_device_eval_batch_size,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    optim="adamw_torch",
    fp16=True,
    learning_rate=CFG.lr,
    warmup_steps=CFG.warmup_steps,
    logging_dir="logs",
    logging_steps=10,
    report_to="none"
)

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    callbacks=[LoggingCallback()],
)

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [19]:
trainer.train()

Epoch,Training Loss,Validation Loss,Acc,Log Loss,Runtime,Samples Per Second,Steps Per Second
1,0.697000,0.693223,0.496697,0.693222,594.274400,8.151000,0.510000
2,0.697200,0.693403,0.507845,0.693405,593.862700,8.157000,0.510000
3,0.697700,0.693082,0.504335,0.693080,600.746100,8.063000,0.504000


Step 20: Loss = 0.6967, All = {'loss': 0.6967, 'grad_norm': 3.047994613647461, 'learning_rate': 1e-05, 'epoch': 0.016, 'step': 10}
Step 30: Loss = 0.6955, All = {'loss': 0.6955, 'grad_norm': 1.8335925340652466, 'learning_rate': 2e-05, 'epoch': 0.032, 'step': 20}
Step 40: Loss = 0.7059, All = {'loss': 0.7059, 'grad_norm': 2.5207717418670654, 'learning_rate': 1.9892183288409707e-05, 'epoch': 0.048, 'step': 30}
Step 50: Loss = 0.6915, All = {'loss': 0.6915, 'grad_norm': 0.9613805413246155, 'learning_rate': 1.978436657681941e-05, 'epoch': 0.064, 'step': 40}
Step 60: Loss = 0.6987, All = {'loss': 0.6987, 'grad_norm': 2.152090072631836, 'learning_rate': 1.9676549865229113e-05, 'epoch': 0.08, 'step': 50}
Step 70: Loss = 0.6976, All = {'loss': 0.6976, 'grad_norm': 1.2096996307373047, 'learning_rate': 1.9568733153638815e-05, 'epoch': 0.096, 'step': 60}
Step 80: Loss = 0.6903, All = {'loss': 0.6903, 'grad_norm': 1.8332371711730957, 'learning_rate': 1.946091644204852e-05, 'epoch': 0.112, 'step': 

TrainOutput(global_step=1875, training_loss=0.6931615804036458, metrics={'train_runtime': 16481.106, 'train_samples_per_second': 1.82, 'train_steps_per_second': 0.114, 'total_flos': 3.973809586176e+16, 'train_loss': 0.6931615804036458, 'epoch': 3.0})

In [ ]:
y_true = test["winner"]
logits = trainer.predict(test).predictions
y_pred_probs = torch.from_numpy(logits).float().softmax(-1).numpy()
acc = accuracy_score(y_true=y_true, y_pred=y_pred_probs.argmax(-1))
print(f"Test - Accuracy: {acc:.4f}") 

joblib.dump(y_pred_probs, f"y_pred_probs_fold_test_acc_{acc:.6f}.pkl")

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report((y_pred_probs[:, 0] <= 0.5).astype(int), y_true, digits=6))

In [ ]:
/(y_pred_probs[:, 0] >= 0.5).astype(int)[:10]

In [ ]:
y_true[:10]